# Image Segmentation


## Load data

### Mounting drive

In [ ]:
# Mount GDrive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Copy and Unzip

In [ ]:
# Copy the zip file and unzip in colab

!cp '/content/drive/My Drive/train_256.zip' train_256.zip
!unzip train_256.zip

## Load CSAIL Pre-Trained model

In [ ]:
# Download the code and pretrained models from CSAIL

%%bash
# Colab-specific setup
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
pip install yacs 2>&1 >> install.log
git init 2>&1 >> install.log
git remote add origin https://github.com/CSAILVision/semantic-segmentation-pytorch.git 2>> install.log
git pull origin master 2>&1 >> install.log
DOWNLOAD_ONLY=1 ./demo_test.sh 2>> install.log

From https://github.com/CSAILVision/semantic-segmentation-pytorch
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master


## Generate Semantic Maps

### Customizations
---


1.   Go to ./config/ade20k-resnet50dilated-ppm_deepsup.yaml and provide a result for TEST path like below 


```
TEST:
  checkpoint: "epoch_20.pth"
  result: "./semantic_maps/"
```
2.   Open the test.py root directory and replace the visualize_result function completly with below code

```
def visualize_result(data, pred, cfg):
    (img, info) = data
 
    # print predictions in descending order
    pred = np.int32(pred)
    pixs = pred.size
    uniques, counts = np.unique(pred, return_counts=True)
    print("Predictions in [{}]:".format(info))
    for idx in np.argsort(counts)[::-1]:
        name = names[uniques[idx] + 1]
        ratio = counts[idx] / pixs * 100
        if ratio > 0.1:
            print("  {}: {:.2f}%".format(name, ratio))
 
    # colorize prediction
    pred_color = colorEncode(pred, colors).astype(np.uint8)
 
    # aggregate images and save
    # commented lines 49 - 50 to prevent concatination <CHANGED>
    # im_vis = np.concatenate((img, pred_color), axis=1)
    # img_name = info.split('/')[-1]
 
    # Added below lines from 53 - 63 <CHANGED>
    im_vis = pred_color
    split_info = info.split('/')
    img_name = split_info[-1]
    category = split_info[-2]
    
    subfolder = False
    if subfolder:
      folder_path = os.path.join(cfg.TEST.result, category)
      if not os.path.isdir(folder_path):
        os.makedirs(folder_path)
    else:
      folder_path = cfg.TEST.result
 
    Image.fromarray(im_vis).save(
        os.path.join(folder_path+"/", img_name.replace('.jpg', '.png')))
```



### Run the test script

In [ ]:
!python -u test.py --imgs '/content/train_256/' | run_log.txt

## Store generated maps

### Zip the folder

In [ ]:
zip -r semantic_maps.zip semantic_maps

### Move the file to drive

In [ ]:
!cp /content/drive/MyDrive/EE599\ Project/Final_semantic_images.zip ./

In [ ]:
import os
os.chdir('/content/drive/MyDrive/EE599_Project/Datasets/train/')

In [ ]:
%pwd

In [ ]:
# !unzip /content/drive/MyDrive/EE599_Project/Datasets/train/Final_semantic_images.zip

## Label Maps generation from Segmentation maps

### Download colors.npy from below link and upload  file to current diretory
[Download here](https://drive.google.com/file/d/1Blbf9nmjqPv_vtX2-sCZ6TqAKXAzgAim/view?usp=sharing)

In [ ]:
import numpy as np
with open('/content/drive/MyDrive/EE599_Project/Notebooks/colors.npy', 'rb') as f:
    colors = np.load(f)

for i,color in enumerate(colors):
    colors[i] = [int(x) for x in color]
    
colors = np.array(colors)
print(colors)

[[120 120 120]
 [180 120 120]
 [  6 230 230]
 [ 80  50  50]
 [  4 200   3]
 [120 120  80]
 [140 140 140]
 [204   5 255]
 [230 230 230]
 [  4 250   7]
 [224   5 255]
 [235 255   7]
 [150   5  61]
 [120 120  70]
 [  8 255  51]
 [255   6  82]
 [143 255 140]
 [204 255   4]
 [255  51   7]
 [204  70   3]
 [  0 102 200]
 [ 61 230 250]
 [255   6  51]
 [ 11 102 255]
 [255   7  71]
 [255   9 224]
 [  9   7 230]
 [220 220 220]
 [255   9  92]
 [112   9 255]
 [  8 255 214]
 [  7 255 224]
 [255 184   6]
 [ 10 255  71]
 [255  41  10]
 [  7 255 255]
 [224 255   8]
 [102   8 255]
 [255  61   6]
 [255 194   7]
 [255 122   8]
 [  0 255  20]
 [255   8  41]
 [255   5 153]
 [  6  51 255]
 [235  12 255]
 [160 150  20]
 [  0 163 255]
 [140 140 140]
 [250  10  15]
 [ 20 255   0]
 [ 31 255   0]
 [255  31   0]
 [255 224   0]
 [153 255   0]
 [  0   0 255]
 [255  71   0]
 [  0 235 255]
 [  0 173 255]
 [ 31   0 255]
 [ 11 200 200]
 [255  82   0]
 [  0 255 245]
 [  0  61 255]
 [  0 255 112]
 [  0 255 133]
 [255   0 

### Create Instance maps directory

In [ ]:
%mkdir Instance_maps
%pwd
! ls

Verify the annotations_dir and Instance_maps names and run the below code

In [ ]:
import glob
images = glob.glob('/content/Final_semantic_images/*.png')
print(len(images))

60000


In [ ]:
list_images = sorted(images)

In [ ]:
list_images[0]

'/content/Final_semantic_images/beach_00000001.png'

In [ ]:
list_images[0]
img = cv2.imread(list_images[0])
img.shape

(256, 256, 3)

In [ ]:
import numpy as np
import random
import os 
import cv2 

from shutil import copy 
from tqdm import tqdm 

annotations_dir = "/content/Final_semantic_images/"
annotations_files = os.listdir(annotations_dir)
annotations_files = [os.path.join(os.path.realpath("."), annotations_dir, x) for x in annotations_files]
annotations_files = sorted(annotations_files)
# print(annotations_files)

for annotation in tqdm(annotations_files):
    # for each file
    img = cv2.imread(annotation)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h,w, _ = img.shape
    
    modified_annotation = np.zeros((h,w))
    
    for i,color in enumerate(colors):
    	# for each class color, i is the index value
        color = color.reshape(1,1,-1)
        mask = (color == img)
        
        r = mask[:,:,0]
        g = mask[:,:,1]
        b = mask[:,:,2]
        
        mask = np.logical_and(r,g)

        mask = np.logical_and(mask, b).astype(np.int64)
        
        mask *= i
        
        modified_annotation += mask
    
    save_path = annotation.replace(annotations_dir, "Instance_map_")
    cv2.imwrite(save_path, modified_annotation)
print('DONE')

In [ ]:
%ls

In [ ]:
img2 = cv2.imread('/content/Instance_maps/beach_00000001.png')
print(img2)

In [ ]:
!zip -r instance_maps.zip /content/Instance_maps
!cp instance_maps.zip /content/drive/MyDrive

In [ ]:
import numpy as np
import random
import os 
import cv2 

from shutil import copy 
from tqdm import tqdm 

annotations_dir = "/content/Final_semantic_images"
annotations_files = os.listdir(annotations_dir)
annotations_files = [os.path.join(os.path.realpath("."), annotations_dir, x) for x in annotations_files]
annotations_files = sorted(annotations_files)
# print(annotations_files)

for annotation in range(len(list_images)):
    # for each file
    img = cv2.imread(list_images[annotation])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h,w, _ = img.shape
    
    modified_annotation = np.zeros((h,w))
    
    for i,color in enumerate(colors):
    	# for each class color, i is the index value
        color = color.reshape(1,1,-1)
        mask = (color == img)
        
        r = mask[:,:,0]
        g = mask[:,:,1]
        b = mask[:,:,2]
        
        mask = np.logical_and(r,g)

        mask = np.logical_and(mask, b).astype(np.int64)
        
        mask *= i
        
        modified_annotation += mask
    path = list_images[annotation]
    save_path = path.replace(annotations_dir, "Instance_map_")
    print(save_path)
    cv2.imwrite(save_path, modified_annotation)
print('DONE')

In [ ]:
% pwd

In [ ]:
cd /content/Instance_maps

In [ ]:
%ls